In [2]:
import pyterrier as pt
if not pt.started():
  print ("not")  
  pt.init()

not
PyTerrier 0.5.0 has loaded Terrier 5.4 (built by craigm on 2021-01-16 14:17)


In [3]:
#!pip install --upgrade fastrank lightgbm 
#!pip install python-terrier

In [4]:
from baseline import read_xml
topics_2020 = read_xml('topics-task-2.xml')

1
[<DOM Element: topics at 0x7f074a03f550>]
50


In [5]:
topics_2020[45]

('46', '\nWhich beverage has more calories per glass: beer or cider?\n')

In [6]:
import requests
#r = requests.post('http://10.30.99.211:8261/gpt_small', data = "What is better for deep learning Python or Matlab?")
#print (r.status_code)
import requests
from xml.dom import minidom
import sys
import argparse

#numpy
import numpy as np

In [7]:
pt.datasets.list_datasets()

,dataset,topics,topics_lang,qrels,corpus,corpus_lang,index,info_url
0,50pct,"[training, validation]",en,"[training, validation]",None,None,"[ex1, ex2]",None
1,antique,"[train, test]",en,"[train, test]",True,en,None,https://ciir.cs.umass.edu/downloads/Antique/re...
2,vaswani,True,en,True,True,en,True,http://ir.dcs.gla.ac.uk/resources/test_collect...
3,trec-deep-learning-docs,"[train, dev, test, test-2020, leaderboard-2020]",en,"[train, dev, test, test-2020]",True,en,None,https://microsoft.github.io/msmarco/
4,trec-deep-learning-passages,"[train, dev, eval, test-2019, test-2020]",en,"[train, dev, test-2019, test-2020]",True,en,None,https://microsoft.github.io/MSMARCO-Passage-Ra...
...,...,...,...,...,...,...,...,...
195,irds:wikir/en1k/validation,True,en,True,True,en,None,https://ir-datasets.com/wikir.html#wikir/en1k/...
196,irds:wikir/en59k,None,None,None,True,en,None,https://ir-datasets.com/wikir.html#wikir/en59k
197,irds:wikir/en59k/test,True,en,True,True,en,None,https://ir-datasets.com/wikir.html#wikir/en59k...
198,irds:wikir/en59k/training,True,en,True,True,en,None,https://ir-datasets.com/wikir.html#wikir/en59k...


Read retireved documents and qrels:

In [8]:
import pickle


with open('/notebook/touche/list_of_un_answ.pkl', 'rb') as f:
    answers_2020 = pickle.load(f)
    
with open('/notebook/touche2021/touche2020-task2-relevance-withbaseline.qrels', 'r') as f:
    qrels_lines = f.readlines()
# you may also want to remove whitespace characters like `\n` at the end of each line
qrels = [x.strip().split() for x in qrels_lines] 


Create qrels dict:

In [9]:
qrels_dict = {}
for elem in qrels:
    query, noninf, docno, rank = elem
    if (query in qrels_dict.keys()):
        qrels_dict[query].append((docno, rank))
    else:
        qrels_dict[query] = []
        qrels_dict[query].append((docno, rank))

In [10]:
qrels_dict['46'][:3]

[('clueweb12-1715wb-31-22014', '0'),
 ('clueweb12-0106wb-15-00715', '0'),
 ('clueweb12-1313wb-74-02250', '0')]

In [11]:
answers_2020['46'][0]

(1644.429,
 'clueweb12-1002wb-27-05922',
 'pca beverage 101 - beer',
 'the finest ciders in north america may well emanate from the province of quebec in canada, which has an established artisinal cider industry with historical ties to normandy cider. these quebecois ciders are slowly starting to appear on the us market. herb-spiced and fruit beers.')

Create proper pandas df documents and qrels

In [13]:
import pandas as pd
import re
info_df = pd.DataFrame(columns=["qid", "query", "docno", "text","label"])
info_df_test = pd.DataFrame(columns=["qid", "query", "docno", "text", "label"])
qrels_df = pd.DataFrame(columns=["qid", "docno", "label"])
qrels_df_test = pd.DataFrame(columns=["qid", "docno", "label"])

for elem in topics_2020:
    qid, query = elem[0], elem[1].strip('\n')
    query = re.sub(r'[^\w\s]','',query)
    qrels = qrels_dict[qid]
    for ind, answer in enumerate(answers_2020[qid]):
        #docno1, label = qrels[ind]
        docno = answer[1]
        score = answer[0]
        text = answer[3]
        label = [el[1] for el in qrels if el[0]==docno]
        
        if (int(qid) < 40 and len(label) > 0):
            df_row = {"qid":qid, "query":query, "docno":docno, "text":text, "label": label[0]}
            info_df = info_df.append(df_row, ignore_index= True)
        else:
            info_df_test = info_df_test.append(df_row, ignore_index= True)
            
    for qrel in qrels_dict[qid]:
        docno, label = qrel
        df_row = {"qid":qid, "query":query, "docno":docno, "labels":label}
        if (int(qid) < 40):
            qrels_df = qrels_df.append(df_row, ignore_index= True)
        else:
            qrels_df_test = qrels_df_test.append(df_row, ignore_index= True)
            


#textscorerTf = pt.text.scorer(body_attr="text", wmodel="Tf")
#rtr = textscorerTf.transform(

AttributeError: type object 'object' has no attribute 'dtype'

In [75]:
print(len(info_df),len(qrels_df))

1120 1367


### Simple ranking

In [77]:
from pyterrier import text

In [78]:
from pyterrier.text import scorer

In [82]:
def create_featured_dataset(some_df):
    textscorerTf = text.scorer(body_attr="text", wmodel='BM25')
    rtr_bm = textscorerTf.transform(some_df)
    textscorerTf = text.scorer(body_attr="text", wmodel='Tf')
    rtr_tf = textscorerTf.transform(some_df)
    textscorerTf = text.scorer(body_attr="text", wmodel='PL2')
    rtr_pl2 = textscorerTf.transform(some_df)
    textscorerTf = text.scorer(body_attr="text", wmodel='DFIC')
    rtr_dfic = textscorerTf.transform(some_df)
    
    rtr_pl2_for_merge = rtr_pl2[['qid', 'docno', 'score', 'labels']]
    rtr_pl2_for_merge = rtr_pl2_for_merge.rename(columns={"score": "score_pl2"})
    
    rtr_tf_for_merge = rtr_tf[['qid', 'docno', 'score', 'labels']]
    rtr_tf_for_merge = rtr_tf_for_merge.rename(columns={"score": "score_tf"})
    
    rtr_bm_for_merge = rtr_bm[['qid', 'docno', 'score','labels']]
    rtr_bm_for_merge = rtr_bm_for_merge.rename(columns={"score": "score_bm"})
    
    rtr_dfic_for_merge = rtr_dfic[['qid', 'docno', 'score','labels']]
    rtr_dfic_for_merge = rtr_dfic_for_merge.rename(columns={"score": "score_dfic"})
    
    result = pd.merge(rtr_pl2_for_merge, rtr_tf_for_merge, on=["qid", "docno"])
    result = pd.merge(result, rtr_bm_for_merge, on=["qid", "docno"])
    result = pd.merge(result, rtr_dfic_for_merge, on=["qid", "docno"])
    zipped = [result["score_pl2"], result["score_tf"], result["score_bm"], result["score_dfic"]]
    unzipped_object = zip(*zipped)
    unzipped_list = list(unzipped_object)
    list_of_features = [np.array(elem) for elem in unzipped_list]
    result['features'] = list_of_features
    return result

#test_ds = create_featured_dataset(info_df_test)
result = create_featured_dataset(info_df)

KeyError: "['labels'] not in index"

In [ ]:
result.head()

## Ranking with LTR

In [160]:
result

,qid,docno,rank,score,query,score_pl2,score_tf,features
0,1,clueweb12-1412wb-36-28142,0,23.759399,What is the difference between sex and love,12.226873,8.0,"(12.226872811709208, 8.0)"
1,1,clueweb12-0200wb-79-18105,1,23.572710,What is the difference between sex and love,12.389736,9.0,"(12.38973604711742, 9.0)"
2,1,clueweb12-1214wb-88-29751,2,23.162551,What is the difference between sex and love,12.102459,10.0,"(12.102459029367102, 10.0)"
3,1,clueweb12-0601wb-52-24897,3,21.569264,What is the difference between sex and love,10.757675,6.0,"(10.757674893691874, 6.0)"
4,1,clueweb12-0706wb-92-20025,4,21.569264,What is the difference between sex and love,10.757675,6.0,"(10.757674893691874, 6.0)"
...,...,...,...,...,...,...,...,...
30986,39,clueweb12-0814wb-82-11198,949,5.294241,Is Java code more readable than code written i...,2.455748,1.0,"(2.4557475851450614, 1.0)"
30987,39,clueweb12-0909wb-01-06662,968,5.187425,Is Java code more readable than code written i...,2.397503,1.0,"(2.3975030894290374, 1.0)"
30988,39,clueweb12-1616wb-95-21533,978,5.084834,Is Java code more readable than code written i...,2.341928,1.0,"(2.3419282603846416, 1.0)"
30989,39,clueweb12-0606wb-13-09259,989,5.084834,Is Java code more readable than code written i...,2.341928,1.0,"(2.3419282603846416, 1.0)"


In [ ]:
for ind, elem in enumerate[]

In [176]:
qrels_df.head()

,qid,docno,label
0,1,clueweb12-0001wb-05-12311,0
1,1,clueweb12-1811wb-62-08424,1
2,1,clueweb12-1811wb-62-08423,1
3,1,clueweb12-1217wb-47-14048,0
4,1,clueweb12-1811wb-62-08425,1


In [20]:
result.head()

,qid,docno,score_pl2,score_tf,score_bm,score_dfic,features
0,1,clueweb12-0200wb-79-18105,13.061090,9.0,25.543436,23.535107,"[13.061089692931793, 9.0, 25.543435870477545, ..."
1,1,clueweb12-1214wb-88-29751,12.789944,10.0,25.162405,23.377776,"[12.789944043049017, 10.0, 25.16240459131109, ..."
2,1,clueweb12-0706wb-92-20025,11.361396,6.0,23.349389,20.876798,"[11.361395517783325, 6.0, 23.349388700158293, ..."
3,1,clueweb12-1400tw-53-02910,8.915299,4.0,19.236397,17.269851,"[8.915298882242885, 4.0, 19.236396596096135, 1..."
4,1,clueweb12-0406wb-58-08558,8.753663,5.0,18.078806,16.233385,"[8.753663308642151, 5.0, 18.07880589092797, 16..."


In [154]:
#bm25 = pt.BatchRetrieve(index, wmodel="BM25")
#tf = pt.BatchRetrieve(index, wmodel="Tf")
#pl2 = pt.BatchRetrieve(index, wmodel="PL2")
#pipeline = (rtr_tf ** rtr_pl2)

In [70]:
def add_ranks1(rtr : pd.DataFrame) -> pd.DataFrame:
    """
        Canonical method for adding a rank column which is calculated based on the score attribute
        for each query. Note that the dataframe is NOT sorted by this operation.
        Arguments
            df: dataframe to create rank attribute for
    """
    rtr.drop(columns=["rank"], errors="ignore", inplace=True)
    if len(rtr) == 0:
        rtr["rank"] = pd.Series(index=rtr.index, dtype='int64')
        return rtr
    print (0)
    # -1 assures that first rank will be FIRST_RANK
    rtr["rank"] = rtr.groupby("qid", sort=False)["score"].rank(ascending=False, method="first").astype(int) -1 + 1
    print (1)
    if True:
        rtr.sort_values(["qid", "rank"], ascending=[True,True], inplace=True)
    return rtr

def transform(model, test_DF):
    """
    Predicts the scores for the given topics.

    Args:
        topicsTest(DataFrame): A dataframe with the test topics.
    """
    test_DF = test_DF.copy()

    # check for change in number of features
    found_numf = test_DF.iloc[0].features.shape[0]
    if model.num_f is not None:
        if found_numf != model.num_f:
            raise ValueError("Expected %d features, but found %d features" % (model.num_f, found_numf))
    if hasattr(model.learner, 'feature_importances_'):
        if len(model.learner.feature_importances_) != found_numf:
            raise ValueError("Expected %d features, but found %d features" % (len(model.learner.feature_importances_), found_numf))

    test_DF["score"] = model.learner.predict(np.stack(test_DF["features"].values))
    return add_ranks1(test_DF)

In [22]:
answ

NameError: name 'answ' is not defined

In [35]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(n_estimators=100)
rf_pipe = pt.ltr.apply_learned_model(rf)
rf_pipe.fit(result, qrels_df)
answs = transform(rf_pipe, result)
#pt.Experiment([result, rf_pipe], result, qrels_df, ["map"], names=["BM25 Baseline", "LTR"])

0
1


In [38]:
def write_qrels(name, rtr):
    qids = rtr['qid']
    Q0s = [0 for elem in qids]
    docs = rtr['docno']
    scores = rtr['score']
    ranks = rtr['rank']
    tags = [name for elem in qids]
    common_list = list(zip(qids, Q0s, docs, ranks, scores, tags))
    with open(name +'.qrels', 'w') as fp:
        fp.write('\n'.join('%s %s %s %s %s %s' % x for x in common_list))
    print ("written " + name +'.qrels')

In [39]:
write_qrels("RF", answs)

written RF.qrels


In [40]:
cord19.get_topics(variant='train').head()

antique-train-queries.txt: 100%|██████████| 133k/133k [00:00<00:00, 250kiB/s]  


,qid,query
0,3097310,what causes severe swelling and pain in the knees
1,3910705,why don t they put parachutes underneath airpl...
2,237390,how to clean alloy cylinder heads
3,2247892,how do i get them whiter
4,1078492,what is cloud 9 and 7th heaven


## LightGMB

In [72]:
result['labels']

KeyError: 'labels'

In [71]:
import lightgbm as lgb

# this configures LightGBM as LambdaMART
lmart_l = lgb.LGBMRanker(
    task="train",
    silent=False,
    min_data_in_leaf=1,
    min_sum_hessian_in_leaf=1,
    max_bin=255,
    num_leaves=31,
    objective="lambdarank",
    metric="ndcg",
    ndcg_eval_at=[10],
    ndcg_at=[10],
    eval_at=[10],
    learning_rate= .1,
    importance_type="gain",
    num_iterations=100,
    early_stopping_rounds=5
)

rf_pipe = pt.ltr.apply_learned_model(lmart_l)
rf_pipe.fit(result, qrels_df)
answs = transform(rf_pipe, result)



ValueError: Should set group for ranking task